# Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [4]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [6]:
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [39]:
class SVM :
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate 
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None
    
    def fit(self, X, y):
        # Convert the labels to the binary representaion (SVM is binary classifier)
        labels = np.unique(y)
        self.labels = labels
        y_binary = np.zeros((len(y), len(labels)))
        for i, label in enumerate(labels):
            y_binary[:,i] = (y==label).astype(int) 
            
        n_samples, n_features = X.shape
        
        # init weights, biases
        self.w = np.zeros((n_features, len(labels )))
        self.b = np.zeros(len(labels))
        
        # Perform gradient descent
        for _ in range(self.n_iters):
            for i in range(n_samples): 
                for j in range(len(labels)):
                    condition = y_binary[i,j]*(np.dot(X[i], self.w[:,j]) - self.b[j])
                    if condition :
                        self.w[:,j] -= self.lr* self.lambda_param * 2 * self.w[:,j]
                    else :
                        self.w[:,j] -= self.lr *(2*self.lambda_param*self.w[:,j] - y_binary[i,j]*X[i])
                        self.b -= self.lr * y_binary[i,j]
            
    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return self.labels[np.argmax(approx, axis=1)]

In [40]:
svm = SVM(lambda_param = 0.01, n_iters = 1000, learning_rate=0.01)
svm.fit(X_train, y_train)

In [41]:
# Make predictions on the test data
y_pred = svm.predict(X_test)

# Compute the accuracy of the model
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.63


In [8]:
y.value_counts()

setosa        50
versicolor    50
virginica     50
Name: species, dtype: int64

### sklearn svm module과 비교하기

In [10]:
rbf = SVC(kernel='rbf', gamma=0.5,C=0.1).fit(X_train, y_train)
poly = SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)


In [11]:
rbf_pred = rbf.predict(X_test)
poly_pred = poly.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')

print('RBF Kernel Accuracy : {:.2f}%'.format(rbf_accuracy*100))
print('RBF Kernel F1 Score : {:.2f}%'.format(rbf_f1*100))

RBF Kernel Accuracy : 86.67%
RBF Kernel F1 Score : 86.79%


In [16]:
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test,poly_pred, average='weighted')

print('Polynomial Kernel Accuracy : {:.2f}%'.format(poly_accuracy*100))
print('Polynomial Kernel F1 Score : {:.2f}%'.format(poly_f1*100))

Polynominal Kernel Accuracy : 86.67%
Polynominal Kernel F1 Score : 86.03%


- sklearn에서는 rbf 커널, polynomial 커널 등을 활용하여 직접 구현한 코드에 비해 Accuracy가 더 높은 것을 확인할 수 있었다.